In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/vs-brats2018/miccai_brats_2018_data_training"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
print(os.listdir("/kaggle/input/model-weight-tc/"))

In [ ]:
# #To remove folders
# import os
# import sys
# import shutil

# LABEL_PATH = "../BraTs_Training_p508_npy/original/Label"

    
# try:
#     shutil.rmtree(LABEL_PATH)
# except OSError as e:
#     print ("Error: %s - %s." % (e.filename, e.strerror))
    
    
    

# DATA_PATH = "../BraTs_Training_p508_npy/original/Data"

    
# try:
#     shutil.rmtree(DATA_PATH)
# except OSError as e:
#     print ("Error: %s - %s." % (e.filename, e.strerror))

    


    
# try:
#     shutil.rmtree("../BraTs_Training_p508_npy/")
# except OSError as e:
#     print ("Error: %s - %s." % (e.filename, e.strerror))

In [ ]:
import os
import sys
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from time import time

IMG_WIDTH = 240
IMG_HEIGHT = 240
IMG_CHANNELS = 4
HW = 100
TRAIN_PATH = "../input/vs-brats2018/miccai_brats_2018_data_training"
STORE_PATH = "../BraTs_Training_p508_npy/original/Data"
LABEL_PATH = "../BraTs_Training_p508_npy/original/Label"


import matplotlib.image as mpimg
from scipy.stats import skew
from scipy.stats import kurtosis
import numpy.ma as ma

dirs_all = os.listdir(TRAIN_PATH)
dirs_all.sort()
dirs_train = dirs_all[:2]
training_data = []
label_data = []


def normalize_image(im):
  im_mask = (im > 0);
  im_compressed_region = ma.array(im,mask=~im_mask).compressed()
  #print(im_compressed_region.size)
  
  #plt.imshow(im_mask)
  #plt.show()
  #plt.imshow(im)
  #plt.show()
  if (im_compressed_region.size == 0):
    return im
  else:
    avg = np.mean(im_compressed_region,dtype='float32')
    std = np.std(im_compressed_region,dtype='float32')
    return (ma.array(im,mask=~im_mask)-avg)/std

axial_data_path = [];
axial_label_path = [];

sagittal_data_path = [];
sagittal_label_path = [];

coronal_data_path = [];
coronal_label_path = [];
for nf, folder in enumerate(dirs_train):
  print("Processing Images...")
  HLG_PATH = os.path.join(TRAIN_PATH,folder)
  hlg_all = os.listdir(HLG_PATH)  
  if nf == 0:
    hlg_all.remove('Brats18_2013_22_1')
    hlg_all.remove('Brats18_CBICA_ABN_1')
    hlg_all.remove('Brats18_TCIA01_201_1')
    hlg_all.remove('Brats18_TCIA08_406_1')
    hlg_all.remove('Brats18_CBICA_ARZ_1')
    hlg_all.remove('Brats18_TCIA01_131_1')
    hlg_all.remove('Brats18_2013_7_1')
  else:
    hlg_all.remove('Brats18_2013_0_1')
    hlg_all.remove('Brats18_2013_6_1')
    hlg_all.remove('Brats18_TCIA10_266_1')  
  np.random.shuffle(hlg_all)
  for n, file in enumerate(hlg_all):
    print("Image: "+ str(nf) + ", " + str(n))
    flair = nib.load(os.path.join(HLG_PATH, file, file + '_flair.nii.gz')).get_data()
    t1 = nib.load(os.path.join(HLG_PATH, file, file + '_t1.nii.gz')).get_data()
    t1ce = nib.load(os.path.join(HLG_PATH, file, file + '_t1ce.nii.gz')).get_data()
    t2 = nib.load(os.path.join(HLG_PATH, file, file + '_t2.nii.gz')).get_data()
    label = nib.load(os.path.join(HLG_PATH, file, file + '_seg.nii.gz')).get_data()
    
    #label_patch = np.zeros([240,240,4],dtype='int')
    print("Axial Data:")    
    folder_directory = os.path.join(STORE_PATH,folder,file,'axial','ori')
    if not os.path.exists(folder_directory):
      os.makedirs(folder_directory)
      for i in range(155):
        if(np.mean(flair[:,:,i]+t1[:,:,i]+t1ce[:,:,i]+t2[:,:,i])!=0):
            data_patch_axial = np.zeros([240,240,4],dtype='float32')

            data_patch_axial[:,:,0] = normalize_image(flair[:,:,i])
            data_patch_axial[:,:,1] = normalize_image(t1[:,:,i])
            data_patch_axial[:,:,2] = normalize_image(t1ce[:,:,i])
            data_patch_axial[:,:,3] = normalize_image(t2[:,:,i])

            np.save(os.path.join(folder_directory,'pao'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'),data_patch_axial)
            axial_data_path.append(os.path.join(folder_directory,'pao'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'));      

    print("Axial Label:")
    folder_directory = os.path.join(LABEL_PATH,folder,file,'axial','ori')
    if not os.path.exists(folder_directory):
      os.makedirs(folder_directory)

      for i in range(155):
        if(np.mean(flair[:,:,i]+t1[:,:,i]+t1ce[:,:,i]+t2[:,:,i])!=0):        
            label_patch_axial = np.zeros([240,240,1],dtype='float32')

            label_patch_axial[:,:,0] = (label[:,:,i]>0.0)*1.0 #WT
            #label_patch_axial[:,:,0] = (label[:,:,i]==1.0) * 1.0 + (label[:,:,i]==4.0)*1.0 #TC
            #label_patch_axial[:,:,0] = (label[:,:,i]==4.0)*1.0 #ET

            np.save(os.path.join(folder_directory,'pao'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'),label_patch_axial)
            axial_label_path.append(os.path.join(folder_directory,'pao'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'));

"""
    ##############################################################################################################################################################################################
    
    print("Sagittal Data:")
    folder_directory = os.path.join(STORE_PATH,folder,file,'sagittal')
    if not os.path.exists(folder_directory):
      os.makedirs(folder_directory)
      for i in range(240):

        data_patch_sagittal = np.zeros([240,240,4],dtype='float32')

        data_patch_sagittal[:,41:196,0] = normalize_image(flair[i,:,:])
        data_patch_sagittal[:,41:196,1] = normalize_image(t1[i,:,:])
        data_patch_sagittal[:,41:196,2] = normalize_image(t1ce[i,:,:])
        data_patch_sagittal[:,41:196,3] = normalize_image(t2[i,:,:])


        np.save(os.path.join(folder_directory,'pso'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'),data_patch_sagittal)
        sagittal_data_path.append(os.path.join(folder_directory,'pso'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'));
    
    print("Sagittal Label:")
    folder_directory = os.path.join(LABEL_PATH,folder,file,'sagittal')
    if not os.path.exists(folder_directory):
      os.makedirs(folder_directory)
      for i in range(240):

        label_patch_sagittal = np.zeros([240,240,1],dtype='int')

        label_patch_sagittal[:,41:196,0] = (label[i,:,:]>0).astype(int)

        np.save(os.path.join(folder_directory,'pso'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'),label_patch_sagittal)
        sagittal_label_path.append(os.path.join(folder_directory,'pso'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'));

    ###############################################################################
    print("Coronal Data:")
    folder_directory = os.path.join(STORE_PATH,folder,file,'coronal')
    if not os.path.exists(folder_directory):
      os.makedirs(folder_directory)
      for i in range(240):

        data_patch_coronal = np.zeros([240,240,4],dtype='float32')

        data_patch_coronal[:,41:196,0] = normalize_image(flair[:,i,:])
        data_patch_coronal[:,41:196,1] = normalize_image(t1[:,i,:])
        data_patch_coronal[:,41:196,2] = normalize_image(t1ce[:,i,:])
        data_patch_coronal[:,41:196,3] = normalize_image(t2[:,i,:])
        
        np.save(os.path.join(folder_directory,'pco'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'),data_patch_coronal)
        coronal_data_path.append(os.path.join(folder_directory,'pco'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'));
          
    print("Coronal Label:")          
    folder_directory = os.path.join(LABEL_PATH,folder,file,'coronal')
    if not os.path.exists(folder_directory):
      os.makedirs(folder_directory)
      for i in range(240):
        
        label_patch_coronal = np.zeros([240,240,1],dtype='int')
        
        label_patch_coronal[:,41:196,0] = (label[:,i,:]>0).astype(int)
        
        np.save(os.path.join(folder_directory,'pco'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'),label_patch_coronal)
        coronal_label_path.append(os.path.join(folder_directory,'pco'+ '_' + str(nf) + '_' + str(n) + '_' + str(i) + '.npy'));
"""   
np.save(os.path.join('../BraTs_Training_p508_npy/axial_data_path'+ '.npy'),axial_data_path)    
np.save(os.path.join('../BraTs_Training_p508_npy/axial_label_path'+ '.npy'),axial_label_path)
#np.save(os.path.join('/content/drive/My Drive/Colab Notebooks/BraTs_Training_p508_npy/sagittal_data_path'+ '.npy'),sagittal_data_path)    
#np.save(os.path.join('/content/drive/My Drive/Colab Notebooks/BraTs_Training_p508_npy/sagittal_label_path'+ '.npy'),sagittal_label_path)
#np.save(os.path.join('/content/drive/My Drive/Colab Notebooks/BraTs_Training_p508_npy/coronal_data_path'+ '.npy'),coronal_data_path)    
#np.save(os.path.join('/content/drive/My Drive/Colab Notebooks/BraTs_Training_p508_npy/coronal_label_path'+ '.npy'),coronal_label_path)

In [ ]:
import os
os.listdir('/kaggle/config')

In [ ]:
# import os
# import numpy as np
# dirs_data = np.load('../BraTs_Training_p508_npy/axial_data_path'+ '.npy')
# dirs_label = np.load('../BraTs_Training_p508_npy/axial_label_path'+ '.npy')
# print(dirs_data)
# print(dirs_label)
# print(dirs_data.shape)
# print(dirs_data.size)
# for i in range(dirs_data.size):
#   print(i)
#   #print(dirs_data[i,])
#   dummy_data = np.load(dirs_data[i,])
#   print(dummy_data.shape)
#   dummy_label = np.load(dirs_label[i,])
#   print(dummy_label.shape)

In [ ]:
import matplotlib.image as mpimg

dirs_all = os.listdir(TRAIN_PATH)
dirs_all.sort()
dirs_train = dirs_all[:2]
training_data = []
label_data = []


import keras
import os
import sys
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from time import time

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam, SGD
from keras import backend as K

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(240,240,4),dim_lab=(240,240,1), n_channels=4,
                 n_classes=2, shuffle=False):
        'Initialization'
        self.dim = dim
        self.dim_lab = dim_lab        
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        #self.image_size = image_size #Thats new
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        #print(indexes)
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size, *self.dim_lab))
        #print(X.shape)
        #print(y.shape)
        # Generate data
        
        
        
        #dirs_data = np.load('/content/drive/My Drive/BraTs_Training_p508_npy/axial_data_path'+ '.npy')
        dirs_data = np.load('../BraTs_Training_p508_npy/axial_data_path'+ '.npy')
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            #X[i,] = np.load(STORE_PATH + "/" + ID)
            #print(ID)
            X[i,] = np.load(ID)
            LABEL_ID=ID.replace("Data", "Label")
            # Store class
            #y[i,] = np.load(LABEL_PATH + "/" + ID)
            y[i,] = np.load(LABEL_ID)

        return X, y

In [ ]:
#print(STORE_PATH)

dirs_all = np.load('../BraTs_Training_p508_npy/axial_data_path'+ '.npy')
#dirs_data = np.load('/content/drive/My Drive/Colab Notebooks/BraTs_Training_p508_npy/axial_data_path'+ '.npy')
#dirs_label = np.load('/content/drive/My Drive/Colab Notebooks/BraTs_Training_p508_npy/axial_label_path'+ '.npy')
#dirs_all = [];
#dirs_all = np.array([dirs_data],[dirs_label])
#print(dirs_all.shape)


#dirs_all.sort()
from keras.preprocessing.image import ImageDataGenerator

#print(dirs_all)

# Parameters
params = {'dim': (240,240,4),
          'dim_lab': (240,240,1),
          'batch_size': 16,
          'n_classes': 2,
          'n_channels': 4,
          'shuffle': False}

# split_len = int(len(dirs_all) * 0.8)
# train = dirs_all[1:split_len]
# validation = dirs_all[split_len:]
# np.random.shuffle(train)
# np.random.shuffle(validation)

np.random.shuffle(dirs_all)
split_len = int(len(dirs_all) * 0.8)
train = dirs_all[1:split_len]
validation = dirs_all[split_len:]

# Generators
training_generator = DataGenerator(train, train, **params)

#print(training_generator)
validation_generator = DataGenerator(validation, validation, **params)

In [ ]:
if(True):
    x,y = training_generator.__getitem__(1);

    print(x.shape)
    print(y.shape)    
    for num in range(16):
        plt.figure(figsize=(15,8))
        plt.subplot(2, 4, 1)
        plt.imshow(x[num,:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 2)
        plt.imshow(x[num,:,:,1],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 3)
        plt.imshow(x[num,:,:,2],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 4)
        plt.imshow(x[num,:,:,3],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 5)
        plt.imshow(y[num,:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 6)
        plt.imshow(y[num,:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 7)
        plt.imshow(y[num,:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.subplot(2, 4, 8)
        plt.imshow(y[num,:,:,0],cmap='gray') # BGR->RGB for plotting
        plt.show()

In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger
from keras.regularizers import l2
from keras.applications.vgg16 import VGG16

IMG_WIDTH = 240
IMG_HEIGHT = 240
IMG_CHANNELS = 4

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)
     
weight_decay=0.0
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))   

#Encoder
# Block 1
conv1 = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1_1', kernel_initializer=initializers.random_normal(stddev=0.01))(inputs)
conv1 = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', kernel_initializer=initializers.random_normal(stddev=0.01))(conv1)
pool1 = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(conv1)

# Block 2
conv2 = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', kernel_initializer=initializers.random_normal(stddev=0.01))(pool1)
conv2 = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', kernel_initializer=initializers.random_normal(stddev=0.01))(conv2)
pool2 = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(conv2)

# Block 3
conv3 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', kernel_initializer=initializers.random_normal(stddev=0.01))(pool2)
conv3 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', kernel_initializer=initializers.random_normal(stddev=0.01))(conv3)
conv3 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', kernel_initializer=initializers.random_normal(stddev=0.01))(conv3)
pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(conv3)

# Block 4
conv4 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', kernel_initializer=initializers.random_normal(stddev=0.01))(pool3)
conv4 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', kernel_initializer=initializers.random_normal(stddev=0.01))(conv4)
conv4 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', kernel_initializer=initializers.random_normal(stddev=0.01))(conv4)
pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(conv4)

# Block 5
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', kernel_initializer=initializers.random_normal(stddev=0.01))(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', kernel_initializer=initializers.random_normal(stddev=0.01))(conv5)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', kernel_initializer=initializers.random_normal(stddev=0.01))(conv5)
pool5 = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(conv5)

#Bottleneck
conv_b = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer=initializers.random_normal(stddev=0.01))(pool5)
conv_b = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer=initializers.random_normal(stddev=0.01))(conv_b)
# drop5 = Dropout(0.5)(conv_b)

#Decoder
deconv_6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv_b)
zero_conv6 = ZeroPadding2D(padding=((1,0),(1,0)))(deconv_6)
up6 = concatenate([zero_conv6, conv5], axis=3)
conv6 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(up6)
conv6 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(conv6)
conv6 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(conv6)

up7 = concatenate([Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv6), conv4], axis=3)
conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(up7)
conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(conv7)
conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(conv7)

up8 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv7), conv3], axis=3)
conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(up8)
conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(conv8)
conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(conv8)

up9 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv8), conv2], axis=3)
conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(up9)
conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(conv9)

up10 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv9), conv1], axis=3)
conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(up10)
conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=initializers.random_normal(stddev=0.01))(conv10)

#Output layer
conv11 = Conv2D(1, (1, 1), activation='sigmoid')(conv10)

model = Model(inputs=[inputs], outputs=[conv11])

weights_path = '/kaggle/input/modelweights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
model.load_weights(weights_path, by_name=True)

# model.summary()
model.compile(optimizer=Adam(lr=1e-4), loss=dice_coef_loss, metrics=[dice_coef])

In [ ]:
#from keras import backend as K
#print(K.image_data_format())
#model = UNet()


#model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
model.summary()

In [ ]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
csv_logger = CSVLogger('training_vgg_WT.csv')
earlystopper = EarlyStopping(patience=20, verbose=1)
checkpointer = ModelCheckpoint('model_BRATS_VGG_WT.h5', verbose=1, save_best_only=True)
lr_callback = ReduceLROnPlateau(monitor='val_dice_coef', patience=3, verbose=1, factor=0.5, min_lr=1e-4)
results = model.fit_generator(generator=training_generator,validation_data=validation_generator, epochs=30, 
                    callbacks=[earlystopper, checkpointer, tensorboard, csv_logger])

In [ ]:
# #Display Result
# trained_model = load_model('/kaggle/Model/model_BRATS_VGG_TC.h5', custom_objects={'dice_coef_loss':
# dice_coef_loss,'dice_coef':                   
# dice_coef})

# for j in range(4):
#   x,y = validation_generator.__getitem__(j);
#   test_data = x[:,:,:,:]
#   pred_value = trained_model.predict(test_data, verbose = 1)
#   acc = 0
#   nn = 0
#   total_acc = 0
#   for i in range(32):    
#     # Some tensor we want to print the value of
#   #   a = dice_coef(y[i,:,:,:],pred_value[i,:,:,:])

#   #   # Add print operation
#   #   a = tf.Print(a, [a], message="This is a: ") 
#     test = np.expand_dims(x[i,:,:,:],axis=0)
#     print(test.shape)
#     test_l = np.expand_dims(y[i,:,:,:],axis=0)
#     score = trained_model.evaluate(test, test_l, batch_size = 1)  
#     total_acc += score[1] 
#     if (score[1]<0.99):
#       acc += score[1]
#       nn += 1
#     print('Score:'+str(score))
#     plt.subplot(1,3,1)
#     plt.imshow(x[i,:,:,0], cmap = 'gray')
#     plt.subplot(1,3,2)
#     plt.imshow(pred_value[i,:,:,0], cmap = 'gray')
#     plt.subplot(1,3,3)
#     plt.imshow(y[i,:,:,0], cmap = 'gray')
#     plt.show()
#   print('i:'+str(j)+' :'+str(acc/nn))  
#   print('i:'+str(j)+' :'+str(total_acc/16)) 

In [ ]:
# import matplotlib.image as mpimg
# from scipy.stats import skew
# from scipy.stats import kurtosis
# import numpy.ma as ma
# from keras.models import *
# from keras.layers import *
# from keras.optimizers import *
# from keras import backend as K
# from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger
# from keras.regularizers import l2
# from keras.applications.vgg16 import VGG16
# from scipy import ndimage
# import keras
# import os
# import sys
# import nibabel as nib
# import numpy as np
# import matplotlib.pyplot as plt

# def dice_coef(y_true, y_pred, smooth=1):
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# def dice_coef_loss(y_true, y_pred):
#     return 1 - dice_coef(y_true, y_pred)
# def normalize_image(im):
#   im_mask = (im > 0);
#   im_compressed_region = ma.array(im,mask=~im_mask).compressed()
#   #print(im_compressed_region.size)
  
#   #plt.imshow(im_mask)
#   #plt.show()
#   #plt.imshow(im)
#   #plt.show()
#   if (im_compressed_region.size == 0):
#     return im
#   else:
#     avg = np.mean(im_compressed_region,dtype='float32')
#     std = np.std(im_compressed_region,dtype='float32')
#     return (ma.array(im,mask=~im_mask)-avg)/std
# TRAIN_PATH = "../input/vs-brats2018/miccai_brats_2018_data_training"
# dirs_all = os.listdir(TRAIN_PATH)
# dirs_all.sort()
# dirs_train = dirs_all[:2]
# trained_model = load_model('/kaggle/input/model-weight-tc/model_BRATS_VGG_WT.h5', custom_objects={'dice_coef_loss': dice_coef_loss,'dice_coef': dice_coef})

# test_patients = ['Brats18_2013_22_1','Brats18_CBICA_ABN_1','Brats18_TCIA01_201_1','Brats18_TCIA08_406_1'
#                 ,'Brats18_CBICA_ARZ_1','Brats18_TCIA01_131_1','Brats18_2013_7_1','Brats18_2013_0_1'
#                 ,'Brats18_2013_6_1','Brats18_TCIA10_266_1']
# count = 0
# dice_test = 0
# for nf, folder in enumerate(dirs_train):
#   print("Processing Images...")
#   HLG_PATH = os.path.join(TRAIN_PATH,folder)
#   hlg_all = os.listdir(HLG_PATH)  

#   for n, file in enumerate(hlg_all):
#     if file in test_patients:
#         count = count + 1
#         print(file)
#         print("Image: "+ str(nf) + ", " + str(n))
#         flair = nib.load(os.path.join(HLG_PATH, file, file + '_flair.nii.gz')).get_data()
#         t1 = nib.load(os.path.join(HLG_PATH, file, file + '_t1.nii.gz')).get_data()
#         t1ce = nib.load(os.path.join(HLG_PATH, file, file + '_t1ce.nii.gz')).get_data()
#         t2 = nib.load(os.path.join(HLG_PATH, file, file + '_t2.nii.gz')).get_data()
#         label = nib.load(os.path.join(HLG_PATH, file, file + '_seg.nii.gz')).get_data()

#         print("Axial Data:")    
#         axial = 0;
#         for i in range(155):
#             if(np.mean(flair[:,:,i]+t1[:,:,i]+t1ce[:,:,i]+t2[:,:,i])!=0):
#                 data_patch_axial = np.zeros([240,240,4],dtype='float32')
#                 data_patch_axial[:,:,0] = normalize_image(flair[:,:,i])
#                 data_patch_axial[:,:,1] = normalize_image(t1[:,:,i])
#                 data_patch_axial[:,:,2] = normalize_image(t1ce[:,:,i])
#                 data_patch_axial[:,:,3] = normalize_image(t2[:,:,i])
#                 if (axial == 0):
#                     data_patch_axial_test = np.expand_dims(data_patch_axial,axis=0)
#                 else:
#                     data_patch_axial = np.expand_dims(data_patch_axial,axis=0)
#                     data_patch_axial_test = np.concatenate((data_patch_axial_test,data_patch_axial),axis = 0)
#                 axial = axial + 1
                
#         axial = 0
#         for i in range(155):
#             if(np.mean(flair[:,:,i]+t1[:,:,i]+t1ce[:,:,i]+t2[:,:,i])!=0):
#                 label_patch_axial = np.zeros([240,240,1],dtype='float32')
                
                
#                 label_patch_axial[:,:,0] = (label[:,:,i]>0.0)*1.0 #WT
#                 #label_patch_axial[:,:,0] = (label[:,:,i]==1.0) * 1.0 + (label[:,:,i]==4.0)*1.0 #TC
#                 #label_patch_axial[:,:,0] = (label[:,:,i]==4.0)*1.0 #ET
# #                 label_patch_axial[:,:,0] = (label[:,:,i]>0.0).astype(int) #WT
# #                 label_patch_axial[:,:,1] = (label[:,:,i]==1).astype(int)
# #                 label_patch_axial[:,:,2] = (label[:,:,i]==2).astype(int)
# #                 label_patch_axial[:,:,3] = (label[:,:,i]==4).astype(int)
#                 if (axial == 0):
#                     label_patch_axial_test = np.expand_dims(label_patch_axial,axis=0)
#                 else:
#                     label_patch_axial = np.expand_dims(label_patch_axial,axis=0)
#                     label_patch_axial_test=np.concatenate((label_patch_axial_test,label_patch_axial),axis = 0)
#                 axial = axial + 1
         
#         print(label_patch_axial_test.shape[0])
#         print(data_patch_axial_test.shape)
#         pred_value = trained_model.predict(data_patch_axial_test, verbose = 1)
#         evaluation = trained_model.evaluate(x=data_patch_axial_test, y=label_patch_axial_test,batch_size = label_patch_axial_test.shape[0])
#         plt.figure(figsize=(8,8)) 
#         plt.imshow(data_patch_axial_test[100,:,:,0], cmap = 'gray')
#         plt.title("Flair")
#         plt.show()
#         plt.figure(figsize=(8,8)) 
#         plt.imshow(data_patch_axial_test[100,:,:,1], cmap = 'gray')
#         plt.title("T1")
#         plt.show()
#         plt.figure(figsize=(8,8)) 
#         plt.imshow(data_patch_axial_test[100,:,:,2], cmap = 'gray')
#         plt.title("T1ce")
#         plt.show()
#         plt.figure(figsize=(8,8)) 
#         plt.imshow(data_patch_axial_test[100,:,:,3], cmap = 'gray')
#         plt.title("T2")
#         plt.show()
#         plt.figure(figsize=(8,8)) 
#         plt.imshow(pred_value[100,:,:,0], cmap = 'gray')
#         plt.title("Predicted")
#         plt.show()
#         plt.figure(figsize=(8,8)) 
#         plt.imshow(label_patch_axial_test[100,:,:,0], cmap = 'gray')
#         plt.title("Ground Truth")
#         plt.show()
#         print(evaluation)
#         dice_test = np.asarray(evaluation) +dice_test

# print("test accuracy:")
# print(dice_test/count)